In [26]:
import pandas as pd
import numpy as np 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.preprocessing import StandardScaler,MinMaxScaler

import seaborn as sns
sns.set(style="darkgrid")
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(11.7,8.27)})

'''Predefined Functions for EDA'''
def eda(data):
    print("----------Top-5- Record----------")
    print(data.head(5))
    print("-----------Information-----------")
    print(data.info())
    print("-----------Data Types-----------")
    print(data.dtypes)
    print("----------Missing value-----------")
    print(data.isnull().sum())
    print("----------Null value-----------")
    print(data.isna().sum())
    print("----------Shape of Data----------")
    print(data.shape)
def graph_insight(data):
    print(set(data.dtypes.tolist()))
    df_num = data.select_dtypes(include = ['float64', 'int64'])
    df_num.hist(figsize=(16, 16), bins=50, xlabelsize=8, ylabelsize=8);
def drop_duplicate(data, subset):
    print('Before drop shape:', data.shape)
    before = data.shape[0]
    data.drop_duplicates(subset,keep='first', inplace=True) #subset is list where you have to put all column for duplicate check
    data.reset_index(drop=True, inplace=True)
    print('After drop shape:', data.shape)
    after = data.shape[0]
    print('Total Duplicate:', before-after)


In [27]:
shops=pd.read_csv('data/predict_future_sales/shops.csv')
train=pd.read_csv('data/predict_future_sales/sales_train.csv')
items=pd.read_csv('data/predict_future_sales/items.csv')
items_categories=pd.read_csv('data/predict_future_sales/item_categories.csv')
test=pd.read_csv('data/predict_future_sales/test.csv')
sample=pd.read_csv('data/predict_future_sales/sample_submission.csv')
items_categories['item_category_name']=items_categories['item_category_name'].astype(str)

In [28]:
items_categories.head()

item_category_name  item_category_id
0  PC - Гарнитуры/Наушники                 0
1         Аксессуары - PS2                 1
2         Аксессуары - PS3                 2
3         Аксессуары - PS4                 3
4         Аксессуары - PSP                 4

In [29]:
shops.head()

shop_name  shop_id
0   !Якутск Орджоникидзе, 56 фран        0
1   !Якутск ТЦ "Центральный" фран        1
2                Адыгея ТЦ "Мега"        2
3  Балашиха ТРК "Октябрь-Киномир"        3
4        Волжский ТЦ "Волга Молл"        4

In [30]:
items_categories['meta-category']=items_categories['item_category_name'].apply(lambda x : x.split("-")[0])

In [31]:
len(items_categories['meta-category'].unique())

21

In [32]:
items_categories.shape

(84, 3)

##### After grouping the category, we reduced the number from 84 to 21 

In [33]:
train['date']=pd.to_datetime(train.date,format='%d.%m.%Y')
train.head()

date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0 2013-01-02               0       59    22154      999.00           1.0
1 2013-01-03               0       25     2552      899.00           1.0
2 2013-01-05               0       25     2552      899.00          -1.0
3 2013-01-06               0       25     2554     1709.05           1.0
4 2013-01-15               0       25     2555     1099.00           1.0

In [34]:
train=train[~train.duplicated()]
train.head()

date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0 2013-01-02               0       59    22154      999.00           1.0
1 2013-01-03               0       25     2552      899.00           1.0
2 2013-01-05               0       25     2552      899.00          -1.0
3 2013-01-06               0       25     2554     1709.05           1.0
4 2013-01-15               0       25     2555     1099.00           1.0

In [35]:
train = train[(train.item_price > 0) & (train.item_price < 300000)]


In [36]:
train.date=train.date.apply(lambda x: x.strftime('%Y-%m'))

In [37]:
train.head()

date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  2013-01               0       59    22154      999.00           1.0
1  2013-01               0       25     2552      899.00           1.0
2  2013-01               0       25     2552      899.00          -1.0
3  2013-01               0       25     2554     1709.05           1.0
4  2013-01               0       25     2555     1099.00           1.0

In [38]:
df = train.groupby(['date','item_id','shop_id']).agg({'item_cnt_day':sum,
                                                     'item_price': 'mean'}).reset_index()
df = df[['date','item_id','shop_id','item_cnt_day']]
df = df.pivot_table(index=['item_id','shop_id'], columns='date',values='item_cnt_day',fill_value=0).reset_index()
df.head()


date  item_id  shop_id  2013-01  2013-02  2013-03  2013-04  2013-05  2013-06  \
0           0       54        0        0        0        0        0        0   
1           1       55        0        0        0        0        0        0   
2           2       54        0        0        0        0        0        0   
3           3       54        0        0        0        0        0        0   
4           4       54        0        0        0        0        0        0   

date  2013-07  2013-08   ...     2015-01  2015-02  2015-03  2015-04  2015-05  \
0           0        0   ...           0        0        0        0        0   
1           0        0   ...           0        0        0        0        0   
2           0        0   ...           0        0        0        0        0   
3           0        0   ...           0        0        0        0        0   
4           0        0   ...           0        0        0        0        0   

date  2015-06  2015-07  2015-08  2015-09  2015-10  
0           0        0        0        0        0  
1           0        0        0        0        0  
2           0        0        0        0        0  
3           0        0        0        0        0  
4           0        0        0        0        0  

[5 rows x 36 columns]

In [39]:
df[df['item_id']==50]

date  item_id  shop_id  2013-01  2013-02  2013-03  2013-04  2013-05  2013-06  \
671        50       25        1        0        0        0        0        0   
672        50       54        1        0        0        0        0        0   

date  2013-07  2013-08   ...     2015-01  2015-02  2015-03  2015-04  2015-05  \
671         0        0   ...           0        0        0        0        0   
672         0        0   ...           0        0        0        0        0   

date  2015-06  2015-07  2015-08  2015-09  2015-10  
671         0        0        0        0        0  
672         0        0        0        0        0  

[2 rows x 36 columns]

In [40]:
df_test = pd.merge(test, df, on=['item_id','shop_id'], how='left')
df_test = df_test.fillna(0)
df_test.head()

ID  shop_id  item_id  2013-01  2013-02  2013-03  2013-04  2013-05  2013-06  \
0   0        5     5037      0.0      0.0      0.0      0.0      0.0      0.0   
1   1        5     5320      0.0      0.0      0.0      0.0      0.0      0.0   
2   2        5     5233      0.0      0.0      0.0      0.0      0.0      0.0   
3   3        5     5232      0.0      0.0      0.0      0.0      0.0      0.0   
4   4        5     5268      0.0      0.0      0.0      0.0      0.0      0.0   

   2013-07   ...     2015-01  2015-02  2015-03  2015-04  2015-05  2015-06  \
0      0.0   ...         2.0      0.0      0.0      0.0      1.0      1.0   
1      0.0   ...         0.0      0.0      0.0      0.0      0.0      0.0   
2      0.0   ...         0.0      0.0      0.0      0.0      3.0      2.0   
3      0.0   ...         0.0      0.0      0.0      0.0      0.0      0.0   
4      0.0   ...         0.0      0.0      0.0      0.0      0.0      0.0   

   2015-07  2015-08  2015-09  2015-10  
0      1.0      3.0      1.0      0.0  
1      0.0      0.0      0.0      0.0  
2      0.0      1.0      3.0      1.0  
3      0.0      1.0      0.0      0.0  
4      0.0      0.0      0.0      0.0  

[5 rows x 37 columns]

In [41]:
## bring the metacategory to the item table 
from sklearn import preprocessing


items = pd.merge(items,items_categories, on=['item_category_id'], how='left')
items=items[['item_id','meta-category']]
le = preprocessing.LabelEncoder()
le.fit(items['meta-category'])
items['meta-category']=le.transform(items['meta-category'])
items.head()

item_id  meta-category
0        0             11
1        1             15
2        2             11
3        3             11
4        4             11

In [42]:
df_test = pd.merge( df_test,items, on=['item_id'], how='left')
df_test.head()

ID  shop_id  item_id  2013-01  2013-02  2013-03  2013-04  2013-05  2013-06  \
0   0        5     5037      0.0      0.0      0.0      0.0      0.0      0.0   
1   1        5     5320      0.0      0.0      0.0      0.0      0.0      0.0   
2   2        5     5233      0.0      0.0      0.0      0.0      0.0      0.0   
3   3        5     5232      0.0      0.0      0.0      0.0      0.0      0.0   
4   4        5     5268      0.0      0.0      0.0      0.0      0.0      0.0   

   2013-07      ...        2015-02  2015-03  2015-04  2015-05  2015-06  \
0      0.0      ...            0.0      0.0      0.0      1.0      1.0   
1      0.0      ...            0.0      0.0      0.0      0.0      0.0   
2      0.0      ...            0.0      0.0      0.0      3.0      2.0   
3      0.0      ...            0.0      0.0      0.0      0.0      0.0   
4      0.0      ...            0.0      0.0      0.0      0.0      0.0   

   2015-07  2015-08  2015-09  2015-10  meta-category  
0      1.0      3.0      1.0      0.0              5  
1      0.0      0.0      0.0      0.0             13  
2      0.0      1.0      3.0      1.0              5  
3      0.0      1.0      0.0      0.0              5  
4      0.0      0.0      0.0      0.0              5  

[5 rows x 38 columns]

In [43]:
df_test = df_test.drop(labels=['ID', 'shop_id', 'item_id'], axis=1)

df_test.head()

2013-01  2013-02  2013-03  2013-04  2013-05  2013-06  2013-07  2013-08  \
0      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
2      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
3      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
4      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   

   2013-09  2013-10      ...        2015-02  2015-03  2015-04  2015-05  \
0      0.0      0.0      ...            0.0      0.0      0.0      1.0   
1      0.0      0.0      ...            0.0      0.0      0.0      0.0   
2      0.0      0.0      ...            0.0      0.0      0.0      3.0   
3      0.0      0.0      ...            0.0      0.0      0.0      0.0   
4      0.0      0.0      ...            0.0      0.0      0.0      0.0   

   2015-06  2015-07  2015-08  2015-09  2015-10  meta-category  
0      1.0      1.0      3.0      1.0      0.0              5  
1      0.0      0.0      0.0      0.0      0.0             13  
2      2.0      0.0      1.0      3.0      1.0              5  
3      0.0      0.0      1.0      0.0      0.0              5  
4      0.0      0.0      0.0      0.0      0.0              5  

[5 rows x 35 columns]

In [44]:
TARGET = '2015-10'
y_train = df_test[TARGET]
X_train = df_test.drop(labels=[TARGET], axis=1)

print(y_train.shape)
print(X_train.shape)
X_train.head()

(214200,)
(214200, 34)


2013-01  2013-02  2013-03  2013-04  2013-05  2013-06  2013-07  2013-08  \
0      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
2      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
3      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
4      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   

   2013-09  2013-10      ...        2015-01  2015-02  2015-03  2015-04  \
0      0.0      0.0      ...            2.0      0.0      0.0      0.0   
1      0.0      0.0      ...            0.0      0.0      0.0      0.0   
2      0.0      0.0      ...            0.0      0.0      0.0      0.0   
3      0.0      0.0      ...            0.0      0.0      0.0      0.0   
4      0.0      0.0      ...            0.0      0.0      0.0      0.0   

   2015-05  2015-06  2015-07  2015-08  2015-09  meta-category  
0      1.0      1.0      1.0      3.0      1.0              5  
1      0.0      0.0      0.0      0.0      0.0             13  
2      3.0      2.0      0.0      1.0      3.0              5  
3      0.0      0.0      0.0      1.0      0.0              5  
4      0.0      0.0      0.0      0.0      0.0              5  

[5 rows x 34 columns]

In [45]:
X_train = X_train.values
X_train = X_train.reshape((214200, 34, 1))

y_train = y_train.values
y_train = y_train.reshape(214200, 1)

print(y_train.shape)
print(X_train.shape)

#X_train[:1]


(214200, 1)
(214200, 34, 1)


In [46]:
X_test = df_test.drop(labels=['2013-01'],axis=1)
X_test = X_test.values
X_test = X_test.reshape((214200, 34, 1))
print(X_test.shape)


(214200, 34, 1)


In [47]:
model = Sequential()
model.add(LSTM(64, input_shape=(34, 1)))
model.add(Dropout(0.3))
model.add(Dense(1))

model.compile(loss='mse',
              optimizer='RMSprop',
              metrics=['mean_squared_error'])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 16,961
Trainable params: 16,961
Non-trainable params: 0
_________________________________________________________________


In [48]:
X_train.shape

(214200, 34, 1)

In [49]:
BATCH = 2000

model.fit(X_train, y_train,
          batch_size=BATCH,
          epochs=10
         )


Epoch 1/10
214200/214200 [==============================] - 44s 206us/step - loss: 30.4389 - mean_squared_error: 30.4389
Epoch 2/10
214200/214200 [==============================] - 42s 197us/step - loss: 30.0802 - mean_squared_error: 30.0802
Epoch 3/10
214200/214200 [==============================] - 43s 201us/step - loss: 29.8158 - mean_squared_error: 29.8158
Epoch 4/10
214200/214200 [==============================] - 43s 201us/step - loss: 29.7358 - mean_squared_error: 29.7358
Epoch 5/10
214200/214200 [==============================] - 42s 197us/step - loss: 29.6535 - mean_squared_error: 29.6535
Epoch 6/10
214200/214200 [==============================] - 42s 198us/step - loss: 29.5658 - mean_squared_error: 29.5658
Epoch 7/10
214200/214200 [==============================] - 43s 199us/step - loss: 29.4334 - mean_squared_error: 29.4334
Epoch 8/10
214200/214200 [==============================] - 44s 204us/step - loss: 29.4164 - mean_squared_error: 29.4164
Epoch 9/10
214200/214200 [======

In [50]:
# Get the test set predictions and clip values to the specified range
y_pred = model.predict(X_test).clip(0., 20.)

# Create the submission file and submit!
preds = pd.DataFrame(y_pred, columns=['item_cnt_month'])
preds.to_csv('submission.csv',index_label='ID')